# Digit Recognition Using K-Nearest Neighbours from Scratch

### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [3]:
train_df = pd.read_csv("data/trainset.csv")
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel244,pixel245,pixel246,pixel247,pixel248,pixel249,pixel250,pixel251,pixel252,pixel253
0,1,0,0,0,0,0,0,0,0,0,...,77,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,243,85,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,253,39,0,0,0,0,0,0,0,0


In [4]:
train_df['label'].value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [5]:
X = train_df.drop('label', axis=1)
y = train_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y)
Knn_X = pd.concat([X_train, y_train], axis=1)

## Algorithm
1. Calculate Euclidean distance between the test samples and each row of the train set
2. Sort the distances by distance value, from lowest to highest
3. Take the distance of the k smallest ones
4. Take the value of target variable for these k train set rows
5. Highest Count target variable is chosen

In [11]:
class KNNClassifier():
    def __init__(self, k=5):
        self.k = k
        
    def euclidean_Distance(self, vec1, vec2):
        vec1, vec2 = np.array(vec1), np.array(vec2)
        distance = 0
        for i in range(len(vec1)-1):
            distance += np.square(vec1[i] - vec2[i])
        return np.sqrt(distance)

    def predict(self, X, X_test):
        ### finding distance between test sample with training samples & Sorting
        distances = []
        for i in range(len(X)):
            dist = self.euclidean_Distance(X.iloc[i, :-1], sample)
            distances.append((X.iloc[i], dist))
        distances.sort(key = lambda x: x[1])

        ### Taking the Nearest K neighbours of test sample and taking the predictions
        kneighbours, predictions = [], {}
        for i in range(self.k):
            kneighbours.append(distances[i][0])
            pred = kneighbours[i][-1]
            if pred in predictions:
                predictions[pred] += 1
            else:
                predictions[pred] = 1
        predSorted = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
        return predSorted[0][0]
    
    def score(self, y_pred, y_true):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        acc = (y_true == y_pred).mean()
        return acc     

In [12]:
knn = KNNClassifier()

preds = []
i = 0
for _, row in X_test.iterrows():
    sample = np.array(row)
    y_pred = knn.predict(Knn_X, sample)
    preds.append(y_pred)
    i += 1
    if (i == 20):
        break
preds = np.array(preds)

In [13]:
no_of_test = 20
pred_df = pd.DataFrame({'Prediction': preds, 'Actual': y_test[:no_of_test]}, 
                       columns=['Actual', 'Prediction'])

In [14]:
pred_df.head()

,Actual,Prediction
39641,8,0
24444,6,6
31918,2,2
9848,9,9
25176,7,7


In [15]:
knn.score(preds, np.array(y_test[:20]))

0.8

* Accuracy is 80% on a test-size of 20
    - since Prediction time is too high, test-size is taken small 

### Sklearn Implementation 

In [16]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

sklearn_pred = model.predict(X_test.iloc[:20])

In [17]:
sklearn_pred

array([0, 6, 2, 9, 7, 9, 4, 1, 0, 0, 9, 0, 5, 0, 1, 9, 4, 1, 1, 5])

In [18]:
model.score(X_test[:20], y_test[:20])

0.8

* Accuracy is 80% on a test-size of 20
    - The implementation from scratch is having same predictions

In [19]:
skpred = pd.DataFrame({'SK-Prediction': preds}, index = X_test.index[:20])

### Comparison With Sklearn Implemention

#### Accuracy

In [20]:
comparison = pd.concat([pred_df, skpred], axis=1)
comparison

,Actual,Prediction,SK-Prediction
39641,8,0,0
24444,6,6,6
31918,2,2,2
9848,9,9,9
25176,7,7,7
32786,7,9,9
21425,9,4,4
10167,1,1,1
18526,0,0,0
1853,8,0,0


* Both Implementation have same predictions and same accuracies
    - Accuracy = 80%

#### Time 

In [21]:
sample = np.array(X_test.iloc[0])

In [22]:
%%time
knn = KNNClassifier()
y_pred = knn.predict(Knn_X, sample)

CPU times: user 26.7 s, sys: 163 ms, total: 26.9 s
Wall time: 26.8 s


In [23]:
sample = sample.reshape(1, -1)

In [24]:
%%time
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
sklearn_pred = model.predict(sample)

CPU times: user 1.78 s, sys: 7.36 ms, total: 1.79 s
Wall time: 1.79 s


* Time taken for implementation from scratch (26.4s for one sample) is much harder than the Sklearn implementation(>5s for all sample).
    - This is because Scikit-learn is highly optimized for performance